In [7]:
%pip install --upgrade pip 
%pip install kaggle PyYAML

Note: you may need to restart the kernel to use updated packages.
  Using cached PyYAML-6.0.2-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (2.1 kB)
Using cached PyYAML-6.0.2-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (751 kB)
Note: you may need to restart the kernel to use updated packages.


In [10]:
import os, json, yaml, requests, subprocess
from kaggle.api.kaggle_api_extended import KaggleApi

In [ ]:
data_dir = "../data"
kaggle = {
    "title": "ANH Dispatch Reports",
    "subtitle": "ANH Dispatch Reports ETL Pipeline",
    "description": "This project contains the ETL pipeline for the ANH Dispatch Reports dataset.",
    "id": "andreschirinos/anh-dispatch-reports",
    "licenses": [
        {
            "name": "CC0-1.0",
            "title": "CC0 1.0",
            "path": "https://creativecommons.org/publicdomain/zero/1.0/",
        }
    ],
    "keywords": [
        "p2p",
        "exchange",
        "data",
        "pipeline",
        "etl",
        "bob",
        "bolivia",
        "cryptocurrency",
        "bitcoin",
        "blockchain",
    ],
    "name": "anh-dispatch-report",
    "version": "1.0.0",
    "contributors": [{"title": "Andres Chirinos", "role": "author"}],
}

In [12]:
file_to_upload = 'data/dispatches.csv'
drive_folder_id = os.getenv('DRIVE_FOLDER_ID')

In [13]:
api = KaggleApi()
api.authenticate()

In [ ]:
#datapackage = os.path.join(data_dir, "datapackage.json")
#with open(datapackage, "w") as f:
#    json.dump(kaggle, f)

metadata_path = os.path.join(data_dir, "dataset-metadata.json")
with open(metadata_path, "w") as f:
    kaggle_metadata = dict()
    kaggle_metadata["title"] = kaggle["title"]
    kaggle_metadata["subtitle"] = kaggle["subtitle"]
    kaggle_metadata["description"] = kaggle["description"]
    kaggle_metadata["id"] = kaggle["id"]
    kaggle_metadata["convertToCsv"] = False
    kaggle_metadata["deleteOldVersions"] = False
    kaggle_metadata["categoryIds"] = kaggle["keywords"]
    kaggle_metadata["isPrivate"] = True
    kaggle_metadata["licenseName"] = kaggle["licenses"][0]["name"]
    kaggle_metadata["slug"] = kaggle["id"].split("/")[1]
    kaggle_metadata["ownerSlug"] = kaggle["id"].split("/")[0]
    files = list()
    for resource in kaggle["resources"]:
        file = dict()
        file["token"] = resource["path"]
        file["description"] = resource["description"]
        columns = list()
        for column in resource["schema"]["fields"]:
            column_dict = dict()
            column_dict["order"] = column["order"]
            column_dict["name"] = column["name"]
            column_dict["type"] = column["type"]
            column_dict["originalType"] = None
            column_dict["description"] = column["description"]
            columns.append(column_dict)
        file["columns"] = columns
        files.append(file)
    kaggle_metadata["files"] = files
    json.dump(kaggle_metadata, f)

In [ ]:
try:
    api.dataset_status(kaggle["id"])
    print("Dataset already exists. Updating...")

    api.dataset_create_version(
        folder=data_dir,
        version_notes="Versión automática generada",
        # quiet=True,
        convert_to_csv=False,
        delete_old_versions=False,
        dir_mode="zip",
    )

except requests.exceptions.HTTPError as e:
    print(e)
    if e.response.status_code in [403,401]:
        print("Dataset not found. Creating a new one...")

        api.dataset_create_new(
            folder=data_dir,
            convert_to_csv=False,
            dir_mode="zip",
            public=True,
            # quiet=True,
        )

In [ ]:
print("Cargado")